In [34]:
#For importing modules
import os, sys

#For file handling
from glob import glob
from os import listdir
from os.path import isfile, join
import json 

#For image processing
import PIL
import cv2
import nibabel as nib
import nibabel.processing 
from nibabel.processing import resample_to_output
import nilearn

#Image visualisation
import matplotlib.pyplot as plt

#Other
import numpy as np
import math
from random import sample

#Modeling
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from nilearn.image import concat_imgs, mean_img, resample_img

import tensorflow as tflow
from tensorflow.keras.layers import Flatten
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import precision_score, recall_score, confusion_matrix,classification_report


In [2]:
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from nilearn.image import concat_imgs, mean_img, resample_img

import tensorflow as tflow
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, BatchNormalization
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import os

import nibabel as nib
import nilearn

import json 

import numpy as np
import math
from random import sample

from sklearn.utils.class_weight import compute_class_weight

In [3]:
sys.path.append('/path/to/2014_07_13_test')
import generate_input

In [5]:
!ls './preprocessed-data/images/'

case_00000.npy case_00060.npy case_00120.npy case_00180.npy case_00240.npy
case_00001.npy case_00061.npy case_00121.npy case_00181.npy case_00241.npy
case_00002.npy case_00062.npy case_00122.npy case_00182.npy case_00242.npy
case_00003.npy case_00063.npy case_00123.npy case_00183.npy case_00243.npy
case_00004.npy case_00064.npy case_00124.npy case_00184.npy case_00244.npy
case_00005.npy case_00065.npy case_00125.npy case_00185.npy case_00245.npy
case_00006.npy case_00066.npy case_00126.npy case_00186.npy case_00246.npy
case_00007.npy case_00067.npy case_00127.npy case_00187.npy case_00247.npy
case_00008.npy case_00068.npy case_00128.npy case_00188.npy case_00248.npy
case_00009.npy case_00069.npy case_00129.npy case_00189.npy case_00249.npy
case_00010.npy case_00070.npy case_00130.npy case_00190.npy case_00250.npy
case_00011.npy case_00071.npy case_00131.npy case_00191.npy case_00251.npy
case_00012.npy case_00072.npy case_00132.npy case_00192.npy case_00252.npy
case_00013.npy case_00073

In [6]:
# x_train, y_train, x_val, y_val, x_test, y_test, id_val, id_test = generate_input.generate_data_input(
#     '../kits21/kits21/data/kits.json', n_samples = 300, neg_pct = 100, rootdir = './preprocessed-data/', 
#     img_dir = 'images', mask_dir = 'masks', val_split = 9, test_split = 81)

Number of labels in labels_dict:  300
Generated list of 300 case IDs of 275 positive and 25 negative labels
Are image and mask path list the same lenght?:  True
Are image and mask dictionaries the same lenght?: True
Are cases and labels lists the same length?:  True 300 300
Generated training, validation and test with 210, 9 and 81 cases.
Are training set and labels lists the same length?:  True 46148 46148
After adding channels:  (46148, 128, 128, 3) (1856, 128, 128, 3) (17160, 128, 128, 3)


In [ ]:
# load validation set
val_case = ['case_00233', 'case_00089', 'case_00050', 'case_00112', 'case_00258', 'case_00246', 'case_00157', 'case_00149','case_00184']
test_case = ['case_00221', 'case_00259', 'case_00087', 'case_00254', 'case_00098', 'case_00023', 'case_00041',
             'case_00080', 'case_00101', 'case_00164', 'case_00002', 'case_00110', 'case_00030', 'case_00068',
             'case_00026', 'case_00063', 'case_00006', 'case_00048', 'case_00250', 'case_00238', 'case_00015',
             'case_00111', 'case_00278', 'case_00133', 'case_00284', 'case_00282', 'case_00269', 'case_00039',
             'case_00033', 'case_00108', 'case_00175', 'case_00161', 'case_00256', 'case_00119', 'case_00286',
             'case_00077', 'case_00162', 'case_00270', 'case_00271', 'case_00285', 'case_00174', 'case_00147',
             'case_00215', 'case_00150', 'case_00052', 'case_00231', 'case_00198', 'case_00117', 'case_00138',
             'case_00211', 'case_00190', 'case_00248', 'case_00235', 'case_00049', 'case_00074', 'case_00107',
             'case_00218', 'case_00001', 'case_00193', 'case_00067', 'case_00289', 'case_00072', 'case_00044',
             'case_00294', 'case_00298', 'case_00263', 'case_00038', 'case_00299', 'case_00249', 'case_00225',
             'case_00217', 'case_00178', 'case_00082', 'case_00035', 'case_00034', 'case_00047', 'case_00276',
             'case_00151', 'case_00226', 'case_00086', 'case_00176']

In [174]:
def generate_part_label_dict(labels_dict, list_of_cases):
    return {k: labels_dict[k] for k in list_of_cases}

def load_data(rootdir, img_dir, json_path, list_cases):
    val_path = generate_input.processed_image_paths(rootdir, img_dir)
    labels_dict = generate_input.malignant_labels_to_dict(json_path)
    labels_dict_short = generate_part_label_dict(labels_dict, list_cases)
    image_dict_short = generate_input.load_nifti_img_and_mask_as_numpy(val_path, list_cases)
    x_set, y_set = generate_input.fill_set(list_cases, labels_dict_short, image_dict_short, labels_dict)
    x_set = generate_input.adding_channel(x_set)
    #image_dict_short = adding_channel_dict(image_dict_short)
    return x_set, y_set

In [197]:
x_val, y_val = load_data('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', val_case)
x_test, y_test = load_data('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', test_case)

In [198]:
all_cases = [ 'case_' + str(number).zfill(5) for number in range(300)]
train_case = [x for x in all_cases if x not in val_case and x not in test_case]
x_train, y_train = load_data('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', train_case)

## upsampling 

In [7]:
from imblearn.over_sampling import SMOTE

x_train_reshape = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])
sm = SMOTE(random_state=64)
x_smote, y_smote = sm.fit_resample(x_train_reshape, y_train)
x_smote = x_smote.reshape(x_smote.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3])

In [8]:
x_smote.shape

(83138, 128, 128, 3)

In [9]:
from sklearn.utils import shuffle
x_smote, y_smote = shuffle(x_smote, y_smote)

## Developing baseline model

In [10]:
from tensorflow.keras.utils import Sequence
import numpy as np   

batch_size = 32
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(x_smote, y_smote, batch_size)


In [11]:
# draft model
#inputs = Input(shape=(224, 224, 3))
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=2)

resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(128,128,3))

for layer in resnet.layers[:170]:
    layer.trainable = False

metrics_list = [tf.keras.metrics.AUC(name = 'auc'),
                tf.keras.metrics.BinaryAccuracy(name = 'accuracy')]

#calculate class weights
class_weights = {0 : 5, 1 : 1} 

optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001, jit_compile = False)
#optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.00002)

x = Flatten()(resnet.output)
x = Dense(1024, activation = 'relu', kernel_regularizer='l2')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(128, activation = 'relu', kernel_regularizer='l2')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(resnet.input, x)

model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= metrics_list)
model.summary()
model.fit(train_gen, validation_data = (x_val, y_val), epochs=5, batch_size = batch_size, class_weight = class_weights)


2023-04-01 20:58:07.971325: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-01 20:58:07.971768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2 Pro
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256    

 conv2_block3_2_conv (Conv2D)   (None, 32, 32, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 32, 32, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 16, 16, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 8, 8, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

...

 conv5_block2_2_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 4, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 4, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

2023-04-01 20:58:09.366280: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-01 20:58:10.842694: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2599/2599 [==============================] - ETA: 0s - loss: 1.1742 - auc: 0.8896 - accuracy: 0.7652

2023-04-01 21:01:40.859671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2599/2599 [==============================] - 215s 82ms/step - loss: 1.1742 - auc: 0.8896 - accuracy: 0.7652 - val_loss: 2.2030 - val_auc: 0.8285 - val_accuracy: 0.2570
Epoch 2/5
2599/2599 [==============================] - 209s 80ms/step - loss: 0.5752 - auc: 0.9703 - accuracy: 0.8976 - val_loss: 0.4277 - val_auc: 0.9265 - val_accuracy: 0.8960
Epoch 3/5
2599/2599 [==============================] - 209s 81ms/step - loss: 0.4706 - auc: 0.9814 - accuracy: 0.9257 - val_loss: 0.8794 - val_auc: 0.6362 - val_accuracy: 0.6509
Epoch 4/5
2599/2599 [==============================] - 210s 81ms/step - loss: 0.4145 - auc: 0.9869 - accuracy: 0.9399 - val_loss: 2.7371 - val_auc: 0.7025 - val_accuracy: 0.2328
Epoch 5/5
2599/2599 [==============================] - 210s 81ms/step - loss: 0.3618 - auc: 0.9900 - accuracy: 0.9504 - val_loss: 0.8181 - val_auc: 0.9487 - val_accuracy: 0.7985


In [27]:
results = model.evaluate(x_val, y_val)
results

58/58 [==============================] - 3s 47ms/step - loss: 0.8181 - auc: 0.9487 - accuracy: 0.7985


[0.818135678768158, 0.9486595392227173, 0.798491358757019]

In [28]:
y_pred = model.predict(x_val)

58/58 [==============================] - 2s 40ms/step


In [30]:
precision = precision_score(y_val, y_pred.round())
recall = recall_score(y_val, y_pred.round())
print(precision)
print(recall)

1.0
0.7861635220125787


In [31]:
confusion_matrix(y_val, y_pred.round())

array([[ 107,    0],
       [ 374, 1375]])

In [32]:
print(classification_report(y_val, y_pred.round()))

              precision    recall  f1-score   support

         0.0       0.22      1.00      0.36       107
         1.0       1.00      0.79      0.88      1749

    accuracy                           0.80      1856
   macro avg       0.61      0.89      0.62      1856
weighted avg       0.96      0.80      0.85      1856



In [18]:
np.unique(y_val, return_counts=True)

(array([0., 1.], dtype=float32), array([ 107, 1749]))

## Test evaluation

In [44]:
y_pred_test = model.predict(x_test)

537/537 [==============================] - 22s 40ms/step


array([[  218,   581],
       [ 4518, 11843]])

In [45]:
confusion_matrix(y_test, y_pred_test.round())

array([[  218,   581],
       [ 4518, 11843]])

In [46]:
print(classification_report(y_test, y_pred_test.round()))

              precision    recall  f1-score   support

         0.0       0.05      0.27      0.08       799
         1.0       0.95      0.72      0.82     16361

    accuracy                           0.70     17160
   macro avg       0.50      0.50      0.45     17160
weighted avg       0.91      0.70      0.79     17160



In [47]:
np.unique(y_test, return_counts=True)

(array([0., 1.], dtype=float32), array([  799, 16361]))

## Save model

In [ ]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model1.h5")

## Load model

In [146]:
# load json and create model
json_file = open('baseline_model/model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("baseline_model/model1.h5")

In [147]:
# evaluate loaded model on test data
optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001, jit_compile = False)
metrics_list = [tf.keras.metrics.AUC(name = 'auc'),
                tf.keras.metrics.BinaryAccuracy(name = 'accuracy')]
loaded_model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= metrics_list)
score1 = loaded_model.evaluate(x_val, y_val)
score1

2023-04-06 13:07:12.869238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


58/58 [==============================] - 4s 52ms/step - loss: 0.8181 - auc: 0.9487 - accuracy: 0.7985


[0.818135678768158, 0.9486595392227173, 0.798491358757019]

## Majority voting

In [164]:
# load validation set
val_case = ['case_00233', 'case_00089', 'case_00050', 'case_00112', 'case_00258', 'case_00246', 'case_00157', 'case_00149','case_00184']
test_case = ['case_00221', 'case_00259', 'case_00087', 'case_00254', 'case_00098', 'case_00023', 'case_00041',
             'case_00080', 'case_00101', 'case_00164', 'case_00002', 'case_00110', 'case_00030', 'case_00068',
             'case_00026', 'case_00063', 'case_00006', 'case_00048', 'case_00250', 'case_00238', 'case_00015',
             'case_00111', 'case_00278', 'case_00133', 'case_00284', 'case_00282', 'case_00269', 'case_00039',
             'case_00033', 'case_00108', 'case_00175', 'case_00161', 'case_00256', 'case_00119', 'case_00286',
             'case_00077', 'case_00162', 'case_00270', 'case_00271', 'case_00285', 'case_00174', 'case_00147',
             'case_00215', 'case_00150', 'case_00052', 'case_00231', 'case_00198', 'case_00117', 'case_00138',
             'case_00211', 'case_00190', 'case_00248', 'case_00235', 'case_00049', 'case_00074', 'case_00107',
             'case_00218', 'case_00001', 'case_00193', 'case_00067', 'case_00289', 'case_00072', 'case_00044',
             'case_00294', 'case_00298', 'case_00263', 'case_00038', 'case_00299', 'case_00249', 'case_00225',
             'case_00217', 'case_00178', 'case_00082', 'case_00035', 'case_00034', 'case_00047', 'case_00276',
             'case_00151', 'case_00226', 'case_00086', 'case_00176']

In [148]:
def adding_channel_dict(image_dict):
    for key in image_dict:
        image_dict[key] = generate_input.adding_channel(image_dict[key])
    return image_dict

def load_data_to_evaluate_by_majority_voting(rootdir, img_dir, json_path, list_cases):
    val_path = generate_input.processed_image_paths(rootdir, img_dir)
    labels_dict = generate_input.malignant_labels_to_dict(json_path)
    labels_dict_short = generate_part_label_dict(labels_dict, list_cases)
    labels_dict_short = {key: float(value) for key, value in labels_dict_short.items()}
    
    image_dict_short = generate_input.load_nifti_img_and_mask_as_numpy(val_path, list_cases)
    image_dict_short = adding_channel_dict(image_dict_short)
    return image_dict_short, labels_dict_short
    
def create_prediction_dict(image_dict, model):
    pred_dict = {}
    for key in image_dict:
        y_pred = model.predict(image_dict[key])
        pred_dict[key] = np.round(np.sum(y_pred)/len(y_pred))
    return pred_dict

def create_true_and_prediction_lists(true_dict, pred_dict):
    common_keys = true_dict.keys() & pred_dict.keys()
    true_lst = []
    pred_lst = [] 
    for key in common_keys:
        true_lst.append(true_dict[key])
        pred_lst.append(pred_dict[key])   
    return true_lst, pred_lst

### validation cases

In [149]:
val_dict, labels_val = load_data_to_evaluate_by_majority_voting('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', val_case)

In [157]:
val_pred_dict = create_prediction_dict(val_dict, loaded_model)

5/5 [==============================] - 0s 45ms/step


In [159]:
val_true_lst, val_pred_lst = create_true_and_prediction_lists(labels_val, val_pred_dict)

In [162]:
print(confusion_matrix(val_true_lst, val_pred_lst))

[[1 0]
 [3 5]]


In [163]:
print(classification_report(val_true_lst, val_pred_lst))

              precision    recall  f1-score   support

         0.0       0.25      1.00      0.40         1
         1.0       1.00      0.62      0.77         8

    accuracy                           0.67         9
   macro avg       0.62      0.81      0.58         9
weighted avg       0.92      0.67      0.73         9



### test cases

In [165]:
test_dict, test_label_dict = load_data_to_evaluate_by_majority_voting('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', test_case)

In [166]:
test_pred_dict = create_prediction_dict(test_dict, loaded_model)

3/3 [==============================] - 0s 137ms/step


In [167]:
test_true_lst, test_pred_lst = create_true_and_prediction_lists(test_label_dict, test_pred_dict)

In [168]:
print(confusion_matrix(test_true_lst, test_pred_lst))

[[ 1  6]
 [17 57]]


In [169]:
print(classification_report(test_true_lst, test_pred_lst))

              precision    recall  f1-score   support

         0.0       0.06      0.14      0.08         7
         1.0       0.90      0.77      0.83        74

    accuracy                           0.72        81
   macro avg       0.48      0.46      0.46        81
weighted avg       0.83      0.72      0.77        81



In [ ]:
### XAI

In [ ]:
# # draft model
# restnet = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3), classes=1)

# # metrics_list = [tf.keras.metrics.SpecificityAtSensitivity(0.7, name = 'specificity'),
# #                tf.keras.metrics.AUC(name = 'auc'),
# #                tf.keras.metrics.FalsePositives(name = 'fp'),
# #                tf.keras.metrics.TrueNegatives(name = 'tn'),
# #                 tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),]

# #calculate class weights
# #class_weights_array = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_train), y = np.concatenate(y_train, axis=0))
# #class_weights = dict(zip(np.unique(y_train), class_weights_array ))


# optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.00002, jit_compile = False)


# model = Sequential()
# model.add(restnet)
# model.add(Dense(224, activation='relu', input_dim=(224, 224, 3)))
# model.add(Dropout(0.3))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= ['accuracy'] )
# model.summary()
# model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=3, batch_size = 32 )


In [ ]:
# # draft model
# #inputs = Input(shape=(224, 224, 3))
# resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

# for layer in resnet.layers:
#     layer.trainable = False

# metrics_list = [tf.keras.metrics.AUC(name = 'auc'),
#                 tf.keras.metrics.BinaryAccuracy(name = 'accuracy')]



# #calculate class weights
# class_weights = {0 : 10, 1 : 1} # 20 is too much put less

# optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.00002, jit_compile = False)


# model = Sequential()
# model.add(resnet)
# model.add(Flatten())
# # model.add(Dense(1024, activation='relu'))
# # model.add(Dropout(0.5))
# # model.add(BatchNormalization())
# # model.add(Dense(512, activation='relu'))
# # model.add(Dropout(0.5))
# # model.add(BatchNormalization())
# # model.add(Dense(256, activation='relu'))
# # model.add(Dropout(0.5))
# # model.add(BatchNormalization())
# # model.add(Dense(128, activation='relu'))
# # model.add(Dropout(0.5))
# # model.add(BatchNormalization())
# # model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))  
# model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= metrics_list)
# model.summary()
# model.fit(train_gen, validation_data = (x_val, y_val), class_weight = class_weights, epochs=10, batch_size = 200)
